In [13]:
import os
#from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import requests     #以后尽可能直接用request，不再用urllib

In [24]:
def openPage(urlPage):
    try:
        headers = {'User-Agent': 'my custom user agent', 'Cookie': 'haha'}    #加headers
        html = requests.get(urlPage, headers = headers)
        #bsObj = BeautifulSoup(html,"html.parser",from_encoding="gb18030")
        bsObj = BeautifulSoup(html.content)
    except Exception as e:
        print('except:', e)
        if e == "HTTP Error 404: Not Found" or e == "HTTP Error 445:":
            print(urlPage)
        else:
            time.sleep(3)
            bsObj = getLinks(urlPage)
    return bsObj

In [25]:
def getLinks(urlpage):
    bsObj = openPage(urlpage)
    weeklist = bsObj.find("div",{"class":"erji_list1"}).find_all("li")
    for li in weeklist:
        link = li.find("a")['href'][1:]
        if li not in week_dict.values():
            week_dict[li.find("a").get_text()] = "http://www.chinaivdc.cn/cnic/zyzx/lgzb" + li.find("a")['href'][1:]

In [27]:
week_dict = {}
url = "http://www.chinaivdc.cn/cnic/zyzx/lgzb/index"
getLinks(url + ".htm")    #首页
for i in range(1,4):     #从第二页开始的页码
    getLinks(url + "_" + str(i) + ".htm")

In [106]:
for j in range(40,53):
    week_dict.pop("2009 第" + str(j) + "周")

In [18]:
print(len(week_dict.values()))

80


In [30]:
ILIdata = []
for k,v in week_dict.items():
    try:
        date = k
        
        headers = {'User-Agent': 'my custom user agent', 'Cookie': 'qwer'}
        res = requests.get(v,headers = headers)
        res.encoding= 'utf-8'
        reg = '<.*?>|&nbsp'
        content = re.sub(reg, "", res.text)                #清理内容，去掉所有HTML标签
        reg_float = '\d+\.\d+'                             #匹配小数，/d+:任意0-多位数字；/.：匹配小数点
        
        #南方
        reg_south = '(南方省份哨点医院|南方省份2005年设立的哨点医院).*?[，]'              #a.*?b:匹配一长段字符中以a开头以b结尾的子串
        south = re.search(reg_south,content).group()                             
        S_ILI = re.search(reg_float, south).group()
        s_ili = float(S_ILI)
        
        #北方
        reg_north = '(北方省份哨点医院|北方省份内科监测诊室|北方省份2005年设立的哨点医院).*?[，]'         
        north = re.search(reg_north,content).group()                          
        N_ILI = re.search(reg_float, north).group()
        n_ili = float(N_ILI)
        
        ILIdata.append([date, s_ili, n_ili])
     
    except Exception as e:
        print(v)
        print('except:', e)
    time.sleep(3)

http://www.chinaivdc.cn/cnic/zyzx/lgzb/201811/t20181105_196899.htm
except: 'NoneType' object has no attribute 'group'
http://www.chinaivdc.cn/cnic/zyzx/lgzb/201810/t20181029_196698.htm
except: 'NoneType' object has no attribute 'group'
http://www.chinaivdc.cn/cnic/zyzx/lgzb/201810/t20181016_195103.htm
except: 'NoneType' object has no attribute 'group'


In [31]:
ILIdata_df = pd.DataFrame(ILIdata, columns = ["time","south_ILI","north_ILI"])
writer = pd.ExcelWriter("C:\\D\\HUST\\research_flu_forecast\\influenza data\\ILI_201001-202012\\ILI_202012.xlsx")
ILIdata_df.to_excel(writer,"sheet1",index = False)
writer.save()

In [ ]:
----------------------------test------------------------------------

In [139]:
url = "http://ivdc.chinacdc.cn/cnic/zyzx/lgzb/201812/t20181224_198439.htm"
res = requests.get(url)
res.encoding= 'utf-8'
reg = '<.*?>|&nbsp'
content = re.sub(reg, "", res.text)                #清理内容，去掉所有HTML标签
reg_float = '\d+\.\d+'                             #匹配小数，/d+:任意0-多位数字；/.：匹配小数点
        
#南方
reg_south = '南方省份哨.*?[，]'              #a.*?b:匹配一长段字符中以a开头以b结尾的子串
south = re.search(reg_south,content).group()                             
S_ILI = re.search(reg_float, south).group()
s_ili = float(S_ILI)
        
#北方
reg_north = '北方省份哨.*?[，]'         
north = re.search(reg_north,content).group()                          
N_ILI = re.search(reg_float, north).group()
n_ili = float(N_ILI)

In [140]:
print(s_ili,n_ili)

3.5 3.4


In [67]:
print(sentence)

南方省份哨点医院ILI%（2.9%）低于2010年第52周（3.0%），


In [60]:
print(content)





中国国家流感中心




<!--
.erji_text1{
	overflow: auto;
	zoom: 1;
	line-height: 25px;
	font-size: 14px;
	color: #000000;
	text-align:left;
	padding:20px 20px;
}
.erji_text1 h3{
	line-height: 25px;
	font-size: 16px;
	color: #000000;
	text-align: center;
	padding:10px;
}
.erji_text1 h5{
	line-height: 25px;
	font-size: 14px;
	color: #000000;
	text-align:left;
	padding:10px 0px;
}
.erji_text1 p{
	line-height: 25px;
	font-size: 14px;
	color: #000000;
	text-align:left;
}


.zw_box {
	overflow:auto;
	zoom:1;
}
.zw_box h2 {
	height: 70px;
	overflow: hidden;
	padding-top: 4px;
	line-height: 70px;
	font-size: 15px;
	font-weight: bold;
	color: #000;
	text-align: left;
	padding-left: 75px;
	background-image: url(../../../../images/m_57_liugan_01.gif);
	background-repeat: no-repeat;
	background-position: left bottom;
}
.path{
	line-height: 45px;
	font-size: 12px;
	color: #000000;
	text-align:left;
	text-indent:25px;
}
.path a{
	line-height: 45px;
	font-size: 12px;
	color: #000000;
}
.path a:hover{
	lin

In [ ]:
南方省份哨点医院ILI%（2.9%）
'a.*?b'就是a开始b结束的匹配
如果要限制是一行的开头和末尾的话
就是
'^a.*?b$
'^a.*?b$' 带问号是贪婪模式，可以匹配多个结果，去掉问号只匹配一个